# Device check
This notebook shows how to make preliminary checks on a device. [mesoscoPy](https://github.com/julienbarrier/mesoscopy) integrates function to perform the following:
* IV curves for all contacts
* gate leak check
* contact gate dependence

We assume here to be working with a Keithley 2600 and an Oxford Triton

## Imports

In [ ]:
%matplotlib inline

from qcodes import initialise_or_create_database_at, load_or_create_experiment
from mesoscopy.instrument.station import init_station
from mesoscopy.instrument.keithley import initialise_keithley
from mesoscopy.instrument.lockin import initialise_lockin
from mesoscopy.experiment.characterise_contacts import contact_IV, test_gate, twoprobe_contacts

## Set up station and database
### set up measurement station

we use the function `init_station` from `mesoscopy.instrument.station`. It is composed of the following arguments:
* `keithley_addr`: Visa address (string)
* `triton_addr`: IP address (stringg)
* `MFLI_num`: number of MFLI (string)
* `current_range`: factor for the input current. the first MFLI sources an AC voltage. We convert this to a current source by adding a resistor in series. `current_range` correspnods to the current sourced with a 1V AC excitation. Optional value.

Example:

In [ ]:
station = init_station("TCPIP::192.168.0.13::inst0::INSTR",
                       "192.168.0.2",
                       "4406","4330", "3550",
                       current_range=1e-9)

### initialise instruments
The keithley can be initialised with voltage compliants limits (`limits_v` in Volts) and maximum sweeping rate (`max_rate` in Volt/sec). These take tuples (or sequences) to account for multiple channels.
The maximum sweeping rate is coded in [mesoscoPy](https://github.com/julienbarrier/mesoscopy)'s functions. It is not shared with the instrument.

In [ ]:
initialise_keithley(station, limits_v=[10, 70], max_rate=[0.12, 0.12])

The lockins can be initialised by setting up the amplitude (`ampl` in Volts) and the frequency (`freq`, in Hz):

In [ ]:
initialise_lockin(station, freq=127,ampl=2)

### Name and initialise database

In [ ]:
initialise_or_create_database_at('../database.db')
exp = load_or_create_experiment(
    experiment_name='test contacts',
    sample_name='no_sample'
)

## test gates
We are going to test the gate. For the following, we shall connect the Keithley's channel A to the gate of interest, and float everything else.

In [ ]:
test_gate('silicon_gate (1)',
          station=station,
          exp=exp,
          sweeprange=60,
          do_plot=True)

In [ ]:
test_gate('graphite_gate (2)',
          station=station,
          exp=exp,
          sweeprange=20,
          do_plot=True)

## test contact IV
we are going to test the contacts, one by one. For the following, we shall connect the Keithley's channel B to the contact of interest, and ground everything else

In [ ]:
contact_IV(3,  # contact number. can be a string. 
           station,  # station containing a station.keithely instrument
           station.triton.T8,  # additional parameters that we want to record
           exp = exp,  # the experiment object to record the contact characterisation. Optional.
           do_plot = True,  # we plot the result
           do_fit = True  # we fit the IV characteristic with a linear model to get he contact resistance
          )